In [8]:
import pandas as pd
import requests
import json
import os
from pathlib import Path

In [9]:
city_bike = pd.read_csv('city_bike.csv')
city_bike.head(5)

,id,latitude,longitude,free_bikes,empty_slots,bike_station
0,1f5c4bbbdda9c2d08239135f4adde68f,41.969090,-87.674237,7,39,0
1,1ad7fb8d86ae4bffa241927486981a37,41.793242,-87.587782,13,6,1
2,5722adf4f567f20396ba64db2aa3952b,41.882830,-87.661206,6,5,2
3,d510712148d2ace2f7efade8e41b4fa4,41.950687,-87.668700,12,3,3
4,7d847e4d2e95d93d23dfb60e811f3888,41.929546,-87.643118,13,0,4


In [10]:
city_bike.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 601 entries, 0 to 600
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            601 non-null    object 
 1   latitude      601 non-null    float64
 2   longitude     601 non-null    float64
 3   free_bikes    601 non-null    int64  
 4   empty_slots   601 non-null    int64  
 5   bike_station  601 non-null    int64  
dtypes: float64(2), int64(3), object(1)
memory usage: 28.3+ KB


In [11]:
city_bike.shape

(601, 6)

In [18]:
N = city_bike.shape[0] 
#N = 10
lat = city_bike['latitude'].loc[0]
log = city_bike['longitude'].loc[0]
latlog = f'{lat},{log}'
df_all = pd.DataFrame() 
#Define the end point URL for the milano city bike network
endpoint = f'https://api.foursquare.com/v3/places/search?ll={latlog}'
api_key = os.environ["FOURSQUARE_API_KEY"]
# Create dictionary for headers
headers = {"Accept": "application/json"}
# Add key with our API KEY
headers['Authorization'] = api_key
for bike_station_index in range(0,N):
    params = {'term':'bar', 'radius': '1000' , 'sort_by' : 'distance' , 'sort_by' : 'rating'} 
    req=requests.get(endpoint, params=params ,headers=headers)
    # proceed only if the status code is 200
    print('The status code is {}'.format(req.status_code)) 
    park_details = json.loads(req.text)
    df2 = pd.json_normalize(park_details['results']) 
    df2['bike_station'] = bike_station_index
    df_all = pd.concat([df_all,df2], ignore_index = True)
    #print(bike_station_index)
    print(df_all[['fsq_id','bike_station']])
    

The status code is 200
                     fsq_id  bike_station
0  5333308c498e832a60af3d55             0
1  4a807b49f964a52069f51fe3             0
2  520c391a11d2d981be88347b             0
3  5a6fafafc62b49492c130013             0
4  535b07c3498eba1a59a21680             0
5  5d4addc07928f300080223b3             0
6  4b67a136f964a520ad582be3             0
7  4b4a7352f964a520038826e3             0
8  4b2b8a46f964a52057b724e3             0
9  5abd5f1697cf5a6f517f6e91             0
The status code is 200
                      fsq_id  bike_station
0   5333308c498e832a60af3d55             0
1   4a807b49f964a52069f51fe3             0
2   520c391a11d2d981be88347b             0
3   5a6fafafc62b49492c130013             0
4   535b07c3498eba1a59a21680             0
5   5d4addc07928f300080223b3             0
6   4b67a136f964a520ad582be3             0
7   4b4a7352f964a520038826e3             0
8   4b2b8a46f964a52057b724e3             0
9   5abd5f1697cf5a6f517f6e91             0
10  5333308c498e832

In [19]:
df_merge = df_all.merge(city_bike, on = 'bike_station')

In [20]:
df_merge

,fsq_id,categories,chains,distance,link,name,timezone,geocodes.main.latitude,geocodes.main.longitude,geocodes.roof.latitude,...,geocodes.drop_off.latitude,geocodes.drop_off.longitude,related_places.parent.fsq_id,related_places.parent.name,bike_station,id,latitude,longitude,free_bikes,empty_slots
0,5333308c498e832a60af3d55,"[{'id': 17069, 'name': 'Grocery Store / Superm...",[{'id': '4ad2fef9-2857-45d6-9194-af1e696123e3'...,64,/v3/places/5333308c498e832a60af3d55,Mariano's Fresh Market,America/Chicago,41.969377,-87.674910,41.969377,...,NaN,NaN,NaN,NaN,0,1f5c4bbbdda9c2d08239135f4adde68f,41.969090,-87.674237,7,39
1,4a807b49f964a52069f51fe3,"[{'id': 13068, 'name': 'American Restaurant', ...",[],42,/v3/places/4a807b49f964a52069f51fe3,Golden Nugget Pancake House,America/Chicago,41.968641,-87.673846,41.968641,...,41.968823,-87.673848,NaN,NaN,0,1f5c4bbbdda9c2d08239135f4adde68f,41.969090,-87.674237,7,39
2,520c391a11d2d981be88347b,"[{'id': 13303, 'name': 'Mexican Restaurant', '...",[],172,/v3/places/520c391a11d2d981be88347b,Primo Chuki's Taqueria,America/Chicago,41.969063,-87.672131,41.969063,...,41.968847,-87.672128,NaN,NaN,0,1f5c4bbbdda9c2d08239135f4adde68f,41.969090,-87.674237,7,39
3,5a6fafafc62b49492c130013,"[{'id': 13035, 'name': 'Coffee Shop', 'icon': ...",[{'id': 'ab4c54c0-d68a-012e-5619-003048cad9da'...,50,/v3/places/5a6fafafc62b49492c130013,Starbucks,America/Chicago,41.969019,-87.675061,41.969019,...,NaN,NaN,5333308c498e832a60af3d55,Mariano's Fresh Market,0,1f5c4bbbdda9c2d08239135f4adde68f,41.969090,-87.674237,7,39
4,535b07c3498eba1a59a21680,"[{'id': 13025, 'name': 'Wine Bar', 'icon': {'p...",[],47,/v3/places/535b07c3498eba1a59a21680,Nob's Wine Bar,America/Chicago,41.969211,-87.674899,41.969211,...,NaN,NaN,NaN,NaN,0,1f5c4bbbdda9c2d08239135f4adde68f,41.969090,-87.674237,7,39
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6005,5d4addc07928f300080223b3,"[{'id': 13039, 'name': 'Deli', 'icon': {'prefi...",[{'id': '3bcb578f-16db-4719-97b8-6fd043cf1bb1'...,159,/v3/places/5d4addc07928f300080223b3,Jersey Mike's,America/Chicago,41.969050,-87.672217,41.969050,...,41.968846,-87.672215,NaN,NaN,600,1ddd2cafbc0a206311cfcc662ead1fd1,41.935581,-87.644042,0,6
6006,4b67a136f964a520ad582be3,"[{'id': 13046, 'name': 'Ice Cream Parlor', 'ic...",[{'id': '16cdfa20-992b-0132-fd75-7a163eb2a6fc'...,109,/v3/places/4b67a136f964a520ad582be3,Baskin-Robbins,America/Chicago,41.968722,-87.672922,41.968722,...,41.968838,-87.672923,NaN,NaN,600,1ddd2cafbc0a206311cfcc662ead1fd1,41.935581,-87.644042,0,6
6007,4b4a7352f964a520038826e3,"[{'id': 13058, 'name': 'Hot Dog Joint', 'icon'...",[],249,/v3/places/4b4a7352f964a520038826e3,Byron's Hot Dog,America/Chicago,41.968693,-87.671235,41.968693,...,41.968855,-87.671237,NaN,NaN,600,1ddd2cafbc0a206311cfcc662ead1fd1,41.935581,-87.644042,0,6
6008,4b2b8a46f964a52057b724e3,"[{'id': 13001, 'name': 'Bagel Shop', 'icon': {...",[{'id': '2cb519f8-883c-4263-860a-cd83325fbb97'...,112,/v3/places/4b2b8a46f964a52057b724e3,Dunkin',America/Chicago,41.968722,-87.672922,41.968722,...,41.968838,-87.672923,NaN,NaN,600,1ddd2cafbc0a206311cfcc662ead1fd1,41.935581,-87.644042,0,6


In [21]:
df_model = df_merge[['fsq_id','free_bikes','bike_station','empty_slots','latitude','longitude','distance']]

In [22]:
df_model

,fsq_id,free_bikes,bike_station,empty_slots,latitude,longitude,distance
0,5333308c498e832a60af3d55,7,0,39,41.969090,-87.674237,64
1,4a807b49f964a52069f51fe3,7,0,39,41.969090,-87.674237,42
2,520c391a11d2d981be88347b,7,0,39,41.969090,-87.674237,172
3,5a6fafafc62b49492c130013,7,0,39,41.969090,-87.674237,50
4,535b07c3498eba1a59a21680,7,0,39,41.969090,-87.674237,47
...,...,...,...,...,...,...,...
6005,5d4addc07928f300080223b3,0,600,6,41.935581,-87.644042,159
6006,4b67a136f964a520ad582be3,0,600,6,41.935581,-87.644042,109
6007,4b4a7352f964a520038826e3,0,600,6,41.935581,-87.644042,249
6008,4b2b8a46f964a52057b724e3,0,600,6,41.935581,-87.644042,112


In [23]:
filepath = Path('foursquar_api.csv')
filepath.parent.mkdir(parents=True, exist_ok=True)

df_model.to_csv(filepath, index=False)
df_model.head(5)

,fsq_id,free_bikes,bike_station,empty_slots,latitude,longitude,distance
0,5333308c498e832a60af3d55,7,0,39,41.96909,-87.674237,64
1,4a807b49f964a52069f51fe3,7,0,39,41.96909,-87.674237,42
2,520c391a11d2d981be88347b,7,0,39,41.96909,-87.674237,172
3,5a6fafafc62b49492c130013,7,0,39,41.96909,-87.674237,50
4,535b07c3498eba1a59a21680,7,0,39,41.96909,-87.674237,47
